There are hundreds of output files from a simulation study. These files have .out extension as the four example files in this activity. In each file, there are 10 rows of the trt_eff and power. The first row of each file contains the value of i and j.

(1) Create a database named output.sqlite. The variables of the database are i, j, trt_eff_size, power. Extract the values of i, j and the trt_eff, power table from all .out files and enter them in a table named power_table in the output.sqlite file.

(2) For all rows with j =3, calculate the average trt_eff and average power.

In [1]:
folder = '/content/drive/MyDrive/stat766/day7/'
folder = 'C:/Users/everyday use/Dropbox/stat766Fall2021/Module1_obtain_data/Lesson2work_with_text_data_and_database/activity2_database/'
import os
allfiles = os.listdir(folder)
#print(allfiles)
outfiles = [file for file in allfiles if '.out' in file]
outfiles

['slurm-11680020.out',
 'slurm-11680029.out',
 'slurm-11680040.out',
 'slurm-11680051.out']

In [4]:
# import the sqlite3 module, make a connection to the target database, and create the table
import sqlite3
conn = sqlite3.connect(folder + 'output.sqlite')
cur = conn.cursor()
sql_command = """CREATE TABLE IF NOT EXISTS power_table (i INTEGER, j INTEGER, trt_eff_size REAL, power REAL)"""
cur.execute(sql_command)

In [2]:
# Using one .out file to figure out how to
# extract the value of i, j, trt_eff, and power from one .out file
file = outfiles[0]
file_connection = open(folder + file, 'r')
for k, line in enumerate(file_connection):
    if k==0:
      first_row = line.split()
      print(first_row)
      print(first_row[3], first_row[6])
    else:
      break

content = file_connection.read()
print('length:', len(content.split('trt_eff power')))
file_connection.close()

content.split('trt_eff power')[1].split('\n')[1:11]

['>', 'i', '=', '68', ';', 'j=', '1']
68 1
length: 2


['1  2.720000   100',
 '2  2.724444   100',
 '3  2.728889   100',
 '4  2.733333   100',
 '5  2.737778   100',
 '6  2.742222   100',
 '7  2.746667   100',
 '8  2.751111   100',
 '9  2.755556   100',
 '10 2.760000   100']

In [3]:
# Collect the code and put them in a function to extract the value of i, j, trt_eff, and power from one .out file

def extract(file):
  file_connection = open(folder + file, 'r')
  for k, line in enumerate(file_connection):
    if k==0:
      first_row = line.split()
      i, j = first_row[3], first_row[6]

    else:
      break

  content = file_connection.read()
  file_connection.close()
  out_table = content.split('trt_eff power')[1].split('\n')[1:11]
  return i, j, out_table

extract(file)


('68',
 '1',
 ['1  2.720000   100',
  '2  2.724444   100',
  '3  2.728889   100',
  '4  2.733333   100',
  '5  2.737778   100',
  '6  2.742222   100',
  '7  2.746667   100',
  '8  2.751111   100',
  '9  2.755556   100',
  '10 2.760000   100'])

In [5]:
#Loop through all the .out files and apply the extract( ) function to each file.
for file in outfiles:
    i, j, outtable = extract(file)
    for row in outtable:
      sql_command2  = """INSERT INTO power_table (i, j, trt_eff_size, power) VALUES (?, ?, ?, ?)"""
      one_row_out_table = row.split()
      # # Note: The extracted values are strings. We need to convert them to numerical values.
      values = (int(i), int(j), float(one_row_out_table[1]), float(one_row_out_table[2]) )
      cur.execute(sql_command2, values)
    conn.commit()

# check to see what is in the power_table
cur.execute('SELECT * FROM power_table')
in_table = cur.fetchall()
print(in_table, '\n', len(in_table))
conn.close()

# Note: We only have four .out files and each contains 10 rows of the trt_eff power output. The resulting database should have 40 rows.
# but if you execulte this cell k times, you will end up having 40 k rows. For example, twice leads to 80 rows.
# If this happens, you'd better delete the table and rerun the code.



[(68, 1, 2.72, 100.0), (68, 1, 2.724444, 100.0), (68, 1, 2.728889, 100.0), (68, 1, 2.733333, 100.0), (68, 1, 2.737778, 100.0), (68, 1, 2.742222, 100.0), (68, 1, 2.746667, 100.0), (68, 1, 2.751111, 100.0), (68, 1, 2.755556, 100.0), (68, 1, 2.76, 100.0), (40, 2, 1.6, 100.0), (40, 2, 1.604444, 100.0), (40, 2, 1.608889, 100.0), (40, 2, 1.613333, 100.0), (40, 2, 1.617778, 100.0), (40, 2, 1.622222, 100.0), (40, 2, 1.626667, 100.0), (40, 2, 1.631111, 100.0), (40, 2, 1.635556, 100.0), (40, 2, 1.64, 100.0), (20, 3, 0.8, 97.0), (20, 3, 0.8044444, 80.0), (20, 3, 0.8088889, 80.0), (20, 3, 0.8133333, 79.0), (20, 3, 0.8177778, 80.0), (20, 3, 0.8222222, 80.0), (20, 3, 0.8266667, 81.0), (20, 3, 0.8311111, 79.0), (20, 3, 0.8355556, 79.0), (20, 3, 0.84, 79.0), (56, 3, 2.24, 100.0), (56, 3, 2.244444, 100.0), (56, 3, 2.248889, 100.0), (56, 3, 2.253333, 100.0), (56, 3, 2.257778, 100.0), (56, 3, 2.262222, 100.0), (56, 3, 2.266667, 100.0), (56, 3, 2.271111, 100.0), (56, 3, 2.275556, 100.0), (56, 3, 2.28, 100

Select all rows from the database such that j = 3. Calculate average trt_eff and average power for these rows.

We stored the values in the database as text. So here we need to convert to numerical value first.

If the amount of data is not too large, we can directly work with the data in python using those rows we selected from the data base.

In [6]:

import numpy as np
data_array = np.array(in_table, dtype ='float')
#print(data_array)
sub_array = data_array[data_array[:,1]== 3.0, :]
print( sub_array)

#compute average of each column
print(np.mean(sub_array, axis =0))

# take out only the average for trt_eff and power
print(np.mean(sub_array, axis =0)[2:])

[[ 20.          3.          0.8        97.       ]
 [ 20.          3.          0.8044444  80.       ]
 [ 20.          3.          0.8088889  80.       ]
 [ 20.          3.          0.8133333  79.       ]
 [ 20.          3.          0.8177778  80.       ]
 [ 20.          3.          0.8222222  80.       ]
 [ 20.          3.          0.8266667  81.       ]
 [ 20.          3.          0.8311111  79.       ]
 [ 20.          3.          0.8355556  79.       ]
 [ 20.          3.          0.84       79.       ]
 [ 56.          3.          2.24      100.       ]
 [ 56.          3.          2.244444  100.       ]
 [ 56.          3.          2.248889  100.       ]
 [ 56.          3.          2.253333  100.       ]
 [ 56.          3.          2.257778  100.       ]
 [ 56.          3.          2.262222  100.       ]
 [ 56.          3.          2.266667  100.       ]
 [ 56.          3.          2.271111  100.       ]
 [ 56.          3.          2.275556  100.       ]
 [ 56.          3.          2.2

We could also do this in the database. SQLite AVG function can be used to calculate the average value of a set of values.


In [7]:
import sqlite3
conn = sqlite3.connect(folder + 'output.sqlite')
cur = conn.cursor()

cur.execute("""SELECT AVG(trt_eff_size) FROM power_table WHERE j = 3 """)
print('average of trt_eff_size for all rows with j =3 \n',   cur.fetchone())

cur.execute("""SELECT AVG(power) FROM power_table WHERE j = 3 """)
print('average of power for all rows with j =3 \n',   cur.fetchone())
conn.close()

average of trt_eff_size for all rows with j =3 
 (1.5399999999999998,)
average of power for all rows with j =3 
 (90.7,)
